In [1]:
! pip install moviepy --user

In [2]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pytube import YouTube
import http.client
# from moviepy.editor import *

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


In [17]:
import os
import time 
import socket
import http.client

from bs4 import BeautifulSoup
from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError 

import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image
from pytube import YouTube

In [20]:
class YouTubeScraper:
    def __init__(self):
        self.wd = webdriver.Chrome('/Users/kimminyoung/Desktop/연구프로젝트/crawling/chromedriver_recent')
        self.scraped_count = 0
        self.path = "./"

    def scroll_down(self):
        scroll_count = 0
        print("[scroll_down(): 스크롤 함수 다운 시작]")
        last_height = self.wd.execute_script("return document.documentElement.scrollHeight")

        while True:
            print(f"[스크롤다운: {scroll_count}]")
            self.wd.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
            scroll_count += 1
            time.sleep(2)
            new_height = self.wd.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    def save(self, url, dir_name, index, query):
        try:
            yt = YouTube(url)
            str = yt.title
            file_name = str.replace(" ", "_")
            print(file_name)

            yt.streams.filter(progressive=True, file_extension='mp4',
                              mime_type="video/mp4").order_by('resolution').desc().first().download(
                f'{query}/video/', f'{query}_{index + 1}.mp4')
            print(f"{index + 1}번째 영상 : {file_name}.mp4 다운로드 완료")

            yt.streams.filter(adaptive=True, file_extension='mp4', mime_type="audio/mp4",
                              only_audio=True).order_by('abr').desc().first().download(
                f'{query}/audio/', f'{query}_{index + 1}.mp4')

            print(f"{index + 1}번째 음성 : {file_name}.mp4 다운로드 완료")

            os.system("ffmpeg -i {} -acodec pcm_s16le -ac 1 -ar 16000 {}/{}.wav".format(
                f'/Users/kimminyoung/Desktop/crawling/{query}/audio/{query}_{index + 1}.mp4',
                f'/Users/kimminyoung/Desktop/crawling/{query}/audio/',
                f'{query}_{index + 1}'))
            print(f"{index + 1}번째 음성 : {file_name}.mp4 -> wav로 변환 완료")

            os.remove(f'{query}/audio/{query}_{index + 1}.mp4')
            print(f"{file_name}.mp4 제거 완료")

        except KeyError as e:
            print(e)
            pass
        except (http.client.IncompleteRead) as e:
            print(e)
            pass

    def click_and_save(self, dir_name, video, index, query):
        try:
            time.sleep(0.5)
            url = video.find_element_by_css_selector('#thumbnail').get_attribute('href')
            self.save(url, dir_name, index, query)
            self.scraped_count += 1

        except HTTPError as e:
            print(e)
            pass

    def scraping(self, dir_name, query):
        try:
            url = f"https://www.youtube.com/c/{query}/videos"
            self.wd.get(url)
            self.wd.maximize_window()
            self.scroll_down()
            time.sleep(1)
            video_list = self.wd.find_elements_by_xpath('//*[@id="items"]/ytd-grid-video-renderer')
        except ZeroDivisionError as e:
            print(e)
            pass

        for index, video in enumerate(video_list):
            try:
                self.click_and_save(dir_name, video, index, query)
            except ElementClickInterceptedException as e:
                print(e)
                self.wd.execute_script("window.scrollTo(0, window.scrollY + 100)")
                time.sleep(1)
                self.click_and_save(dir_name, video, index, query)
            except NoSuchElementException as e:
                print(e)
                self.wd.execute_script("window.scrollTo(0, window.scrollY + 100)")
                time.sleep(1)
                self.click_and_save(dir_name, video, index, query)
            except ConnectionResetError as e:
                print(e)
                pass
            except URLError as e:
                print(e)
                pass
            except socket.timeout as e:
                print(e)
                pass
            except socket.gaierror as e:
                print(e)
                pass
            except ElementNotInteractableException as e:
                print(e)
                break

        try:
            print("[스크래핑 종료 (성공률: %.2ff%%)]" % (self.scraped_count / len(video_list) * 100.0))
        except ZeroDivisionError as e:
            print(e)
            pass

        self.wd.close()

In [ ]:
socket.setdefaulttimeout(100)  # 너무 오래 걸릴 경우 끊어야하므로 최대 30초로 지정
youtube_scraper = YouTubeScraper()
query = input("검색어 입력: ")
dir_name = youtube_scraper.path + query
os.makedirs(dir_name)
print(f"[{dir_name}디렉토리 생성]")
youtube_scraper.scraping(dir_name, query)

In [ ]:
# koriri